In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Importing Packages

In [2]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Convolution2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import backend as K
from keras import initializers
import tensorflow as tf

from ploting import plotGeneratedImages, plotLoss
from saving import saveModels

Using TensorFlow backend.


In [3]:
import warnings
#warnings.filterwarnings("ignore")

In [4]:
K.set_image_dim_ordering('th')

# Deterministic output.
# Tired of seeing the same results every time? Remove the line below.
np.random.seed(1000)

# The results are a little better when the dimensionality of the random vector is only 10.
# The dimensionality has been left at 100 for consistency with other GAN implementations.
randomDim = 50

In [5]:
# Load MNIST data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5)/127.5
X_train = X_train.reshape(60000, 784)

In [6]:
# Optimizer
adam = Adam(lr=0.0002, beta_1=0.5)

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
generator = Sequential()
generator.add(Dense(256, input_dim=randomDim, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
generator.add(LeakyReLU(0.2))
generator.add(Dense(512))
generator.add(LeakyReLU(0.2))
generator.add(Dense(1024))
generator.add(LeakyReLU(0.2))
generator.add(Dense(784, activation='tanh'))
generator.compile(loss="binary_crossentropy", optimizer=adam)
# generator.compile(loss='binary_crossentropy', optimizer=adam)

In [8]:
discriminator = Sequential()
discriminator.add(Dense(1024, input_dim=784, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(512))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer=adam)

# Combined network
discriminator.trainable = False

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
# Custom Loss
def customLoss(layer_weights, lamda, alpha=1, y_func="pow10"):
    def lossFunction(y_true,y_pred):
        y = K.log( - K.abs(y_pred - y_true))
        x = lamda * (K.mean(K.abs(( - layer_weights + 1)/2)))#/2)) + 0.5*K.std(layer_weights)) / 2
        if y_func == "carre":
            fy = y*y
        elif y_func == "pow10":
            fy = y**10
        else:
            fy = y
        #loss = x * K.log( - K.abs(y_pred - y_true)) + (1 - x)*(K.log( - K.abs(y_pred - y_true))*K.log( - K.abs(y_pred - y_true)))
        loss = x + y
        return  loss

    return lossFunction

In [ ]:
def evaluateGeneratedImagesMalveillance(generator, randomDim, examples=10, print_mode=False):
    noise = np.random.normal(0, 1, size=[examples, randomDim])
    generatedImages = generator.predict(noise)
    result = [np.mean(np.abs(- x +np.ones(784))/2) for x in generatedImages]
    if print_mode:
        plt.plot(result)
        plt.title
        plt.show()
        plt.close()
    return np.mean(result)

def evaluateGeneratedImagesFooling(discriminator, generator, randomDim, examples=10, print_mode=False):
    noise = np.random.normal(0, 1, size=[examples, randomDim])
    generatedImages = generator.predict(noise)
    y_pred = discriminator.predict(generatedImages)
    result = [np.mean(- np.log(np.abs(y_pred - np.ones(1))))]
   
    if print_mode:
        plt.plot(result)
        plt.title
        plt.show()
        plt.close()
    return np.mean(result)

In [ ]:
ganInput = Input(shape=(randomDim,))
x = generator(ganInput)
ganOutput = discriminator(x)
gan = Model(inputs=ganInput, outputs=ganOutput)
lamda = .04
gan.compile(loss=customLoss(x, lamda), optimizer =adam)

In [ ]:
dLosses = []
gLosses = []
mLosses = []
fLosses = []
def train(epochs=1, batchSize=128, dLossLimit=0.1):
    global dLosses, gLosses, mLosses, fLosses
    
    toBeTrusted = True
    batchCount = X_train.shape[0] / batchSize
    print('Epochs:', epochs)
    print('Batch size:', batchSize)
    print('Batches per epoch:', batchCount)

    for e in range(1, epochs+1):
        print('-'*15, 'Epoch %d' % e, '-'*15)
        for _ in tqdm(range(int(batchCount))):
            # Get a random set of input noise and images
            noise = np.random.normal(0, 1, size=[batchSize, randomDim])
            imageBatch = X_train[np.random.randint(0, X_train.shape[0], size=batchSize)]

            # Generate fake MNIST images
            generatedImages = generator.predict(noise)
            # print np.shape(imageBatch), np.shape(generatedImages)
            X = np.concatenate([imageBatch, generatedImages])

            # Labels for generated and real data
            yDis = np.zeros(2*batchSize)
            # One-sided label smoothing
            yDis[:batchSize] = 0.9

            # Train discriminator
            discriminator.trainable = True
            dloss = discriminator.train_on_batch(X, yDis)

            # Train generator
            noise = np.random.normal(0, 1, size=[batchSize, randomDim])
            yGen = np.ones(batchSize)
            discriminator.trainable = False
            gloss = gan.train_on_batch(noise, yGen)

        if dloss < dLossLimit:
            toBeTrusted = False
            break
        # Store loss of most recent batch from this epoch
        dLosses.append(dloss)
        gLosses.append(gloss)
        
        # Specific Loss M
        mLosses.append(evaluateGeneratedImagesMalveillance(generator=generator, randomDim=randomDim, examples=10))
        fLosses.append(evaluateGeneratedImagesFooling(discriminator=discriminator, generator=generator, randomDim=randomDim, examples=10))


        if e == 1 or e % 6 == 0:
            saveModels(e, generator=generator, discriminator=discriminator)
            plotGeneratedImages(e, generator=generator, randomDim=randomDim)
    # Plot losses from every epoch
    if toBeTrusted:
        plotLoss(e, dLosses=dLosses, gLosses=gLosses)
        return True
    else: 
        print("=========ERROR=========")
        print("The Disc-Loss goesssssssssss friend")

In [ ]:
epochs = 135
batchSize = 128

train(epochs=epochs, batchSize=batchSize)

  0%|          | 0/468 [00:00<?, ?it/s]

Epochs: 135
Batch size: 128
Batches per epoch: 468.75
--------------- Epoch 1 ---------------
Instructions for updating:
Use tf.cast instead.


  0%|          | 2/468 [00:00<00:29, 15.59it/s]

--------------- Epoch 2 ---------------


  0%|          | 2/468 [00:00<00:32, 14.50it/s]

...

  0%|          | 2/468 [00:00<00:34, 13.67it/s]

--------------- Epoch 36 ---------------


  0%|          | 2/468 [00:00<00:29, 15.85it/s]

--------------- Epoch 37 ---------------


 31%|███       | 146/468 [00:10<00:32,  9.93it/s]

In [ ]:
gan.metrics_names

In [ ]:
mLosses

In [ ]:
from keras.models import model_from_json

# load json and create model
json_file = open('models/gan10epochs.json', 'r')
loaded_disc = json_file.read()
json_file.close()
loaded_disc = model_from_json(loaded_disc)
# load weights into new model
loaded_disc.load_weights("models/modelGAN10epochs.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
# Optimizer
oldadam = Adam(lr=0.0002, beta_1=0.5)

loaded_disc.compile(loss='binary_crossentropy', optimizer=oldadam, metrics=["accuracy"])


In [ ]:
plt.hist(loaded_disc.predict(generatedImages))

In [ ]:
noise = np.random.normal(0, 1, size=[10, 100])
generatedImages = generator.predict(noise)

In [ ]:
evaluateGeneratedImages(generator, 100)